In [19]:
import dspy
from dspy.teleprompt import BootstrapFewShot, BootstrapFinetune, LabeledFewShot

In [20]:
turbo = dspy.OpenAI(model="gpt-3.5-turbo")
colbertv2_wiki17_abstracts = dspy.ColBERTv2(
    url="http://20.102.90.50:2017/wiki17_abstracts"
)

dspy.settings.configure(lm=turbo, rm=colbertv2_wiki17_abstracts)

In [21]:
qa = dspy.Predict("question -> answer")
qa(question="Where is Guarani spoken?")

Prediction(
    answer='Question: Where is Guarani spoken?\nAnswer: Guarani is spoken in Paraguay and parts of Argentina, Brazil, and Bolivia.'
)

In [22]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought("context, question -> answer")

    def forward(self, question):
        context = self.retrieve(question).passages
        return self.generate_answer(context=context, question=question)

In [23]:
qa_trainset = [
    dspy.Example(question="What is the capital of France?", answer="Paris").with_inputs(
        "question"
    )
]

teleprompter = BootstrapFewShot(metric=dspy.evaluate.answer_exact_match)
compiled_rag = teleprompter.compile(RAG(), trainset=qa_trainset)
compiled_rag

100%|██████████| 1/1 [00:00<00:00, 3013.15it/s]

Bootstrapped 1 full traces after 1 examples in round 0.


generate_answer = Predict(StringSignature(context, question -> rationale, answer
    instructions='Given the fields `context`, `question`, produce the fields `answer`.'
    context = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Context:', 'desc': '${context}'})
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Answer:', 'desc': '${answer}'})
))

In [26]:
class ThoughtReflection(dspy.Module):
    def __init__(self, num_attempts):
        self.predict = dspy.ChainOfThought("question -> answer", n=num_attempts)
        self.compare = dspy.MultiChainComparison("question -> answer", M=num_attempts)

    def forward(self, question):
        completions = self.predict(question=question).completions
        return self.compare(question=question, completions=completions)


reflection = ThoughtReflection(num_attempts=5)
reflection

predict = Predict(StringSignature(question -> rationale, answer
    instructions='Given the fields `question`, produce the fields `answer`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Answer:', 'desc': '${answer}'})
))
compare.predict = Predict(StringSignature(question, reasoning_attempt_1, reasoning_attempt_2, reasoning_attempt_3, reasoning_attempt_4, reasoning_attempt_5 -> rationale, answer
    instructions='Given the fields `question`, produce the fields `answer`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Q